# 標準的な「あみだくじ」でどの筋を選ぶとどの筋に至るかを数える
### Python で組んだものを Claude にお願いして Julia に書き換えて貰った
数え方の要領は, 「あみだくじ」は「行きつ戻りつ」と1:1に対応する（らしい）事を利用している。

例えば, 筋は5本で横線が4本とすれば, $5+4\times2−1$ 回の移動で, 筋から隣の筋への移動をして, 左端から右端まで動くことを「行きつ戻りつ」と表現しよう。
筋を左から 0,1,2,3,4 とでもすれば, 例えば 0→1→0→1→0→1→0→1→0→1→2→3→4 のように5つの筋を「行きつ戻りつ」して最終的に到達するということ。
確かに「戻る」動き(この例では1→0しかない)が4回あることが確認できる。そしてこの「戻る」動きがあみだくじの置換に対応するというアイデアなのだ。

プログラムの概要は以下の通り。筋の数は tn 横棒の数は an としている。
+ 「行きつ戻りつ」を探るために2進数を総当たりして, 行きつ戻りつで辿り着くものを拾う
  - 先ず2進数を順に当たる
  - 文字列化して 0,1 の配列に直し 各項を2倍して1引くことで, -1,1の配列にする
  - その上で配列の先頭からの部分和が 0以上で tn 以下であるもので, 全体和が tn となる（つまり最後では端に到達する）ものを拾う
+ 拾えたらその「行きつ戻りつ」でどの順に筋を経由するかのリストを作成してそこから「あみだくじ」の置換を拾う
  - 「行きつ戻りつ」の行程で「戻る」ときにその2筋を交換していることになる
  - 筋の経由リストから「戻る」部分をピックアップして, 筋の置換を列挙した配列を作る
  - できた置換の配列で順に置換することで, 選んだ筋と結果到達する筋のリストを作成する
+ 「選んだ筋」と「到達する筋」の2次元配列に該当するものを足していく
+ 最終的に分布が得られる。

In [4]:
function calculate_amidakuji_distribution(tn,an)
#    tn = 5#8
#    an = 4#12
    ln = 2*an+tn
    cnt = 0
    
    println("--start---------------------------------------")
    println("$tn $an")
    
    stat = zeros(Int, tn, tn)
    
    for n in 0:(2^ln-1)
        # Convert n to binary representation of length ln
        binary = digits(n, base=2, pad=ln)
        reverse!(binary)  # Julia's digits returns in reverse order
        
        # Convert 0,1 to -1,1
        lst = [2*x-1 for x in binary]
        
        chk = 0
        for j in 1:ln
            running_sum = sum(lst[1:j-1])
            if running_sum < 0 || running_sum > tn-1
                chk = 1
                break
            end
        end
        
        if chk == 0
            if sum(lst) == tn
                plst = []
                for j in 1:ln
                    push!(plst, sum(lst[1:j-1]))
                end
                
                exclst = []
                for j in 1:(ln-1)
                    if plst[j] > plst[j+1]
                        push!(exclst, [plst[j+1], plst[j]])
                    end
                end
                
                orgl = collect(0:(tn-1))
                for excp in exclst
                    orgl[excp[1]+1], orgl[excp[2]+1] = orgl[excp[2]+1], orgl[excp[1]+1]
                end
                
                for i in 1:tn
                    stat[i, orgl[i]+1] += 1
                end
                
                cnt += 1
            end
        end
    end
    
    println("--Ans-------------------------------------------")
    println(cnt)
    for ii in 1:tn
        println(stat[ii, :])
    end
    println("--end-------------------------------------------")
end

calculate_amidakuji_distribution (generic function with 2 methods)

In [12]:
calculate_amidakuji_distribution(8,10)

--start---------------------------------------
8 10
--Ans-------------------------------------------
1282735
[764877, 279584, 133631, 64604, 27257, 9481, 2693, 608]
[279584, 478114, 262307, 147260, 72988, 29809, 9980, 2693]
[133631, 262307, 365985, 252938, 153368, 75216, 29809, 9481]
[64604, 147260, 252938, 314118, 250202, 153368, 72988, 27257]
[27257, 72988, 153368, 250202, 314118, 252938, 147260, 64604]
[9481, 29809, 75216, 153368, 252938, 365985, 262307, 133631]
[2693, 9980, 29809, 72988, 147260, 262307, 478114, 279584]
[608, 2693, 9481, 27257, 64604, 133631, 279584, 764877]
--end-------------------------------------------


In [14]:
calculate_amidakuji_distribution(8,12)

--start---------------------------------------
8 12
--Ans-------------------------------------------
16131656
[9188341, 3508269, 1778834, 939616, 451633, 184261, 63000, 17702]
[3508269, 5568480, 3238722, 1961381, 1086206, 507952, 197646, 63000]
[1778834, 3238722, 4168532, 3074842, 2048283, 1130230, 507952, 184261]
[939616, 1961381, 3074842, 3550353, 3019342, 2048283, 1086206, 451633]
[451633, 1086206, 2048283, 3019342, 3550353, 3074842, 1961381, 939616]
[184261, 507952, 1130230, 2048283, 3074842, 4168532, 3238722, 1778834]
[63000, 197646, 507952, 1086206, 1961381, 3238722, 5568480, 3508269]
[17702, 63000, 184261, 451633, 939616, 1778834, 3508269, 9188341]
--end-------------------------------------------
